# Tutorial 2: Machine Learning

## How to use this tutorial
  * Select "run all cells" on this notebook from the Run menu in Jupyter notebook or Jupyter
  lab. This step will produce intermediate data output and charts.
  * Some cells print out a url, which you can click on and bring up an interactive web UI to
    visualize the graph data.
  * In the unlikely event that the notebook becomes irresponsive, you can try "Restart
    Kernel" from the Kernel menu, then run individual cells one by one using `Shift+Enter`.
  * Some tutorials use local clusters consisting of multiple processes to mimic the effects
  of graph distribution over a remote clusters. By default, these local clusters
  automatically stop after idling for 15min to conserve CPU and memory resources. You will
  need to rerun the entire notebook if your local cluster stopped due to inactivity.
  * Additional resources (video demos & blogs) are available at http://juliustech.co
  * To report any issues, get help or request features, please raise an issue at:
   https://github.com/JuliusTechCo/JuliusGraph/issues

## Introduction

This Tutorial shows how to use Julius Graph Engine to set up the training and validation of
a machine learning model. We implement the classic example of using several different ML
models to predict (or postdict) the survival of Titanic passengers by using the well-known
Titanic dataset.

## Data Exploration, Visualization and Cleaning

Data scientits can use Julius Graph Engine to quickly explore and visualize data
from different sources. Julius provides connections to many data sources and formats,
such as CSV, web url, relational Databases, various NoSQL Databases, etc.

`DataScience` package is part of Julius distribution, it provides a rich set of
rules and atoms for data sourcing, cleansing, and machine learning. In this notebook,
we will show how the rules and atoms defined in `DataScience` to quickly build the ML
pipeline for the Titanic data set.

The relevant rules from `DataScience` package for this tutorial are listed below
for your reference.

```julia
@addrules ds begin
    dfsrc(df::DataFrame) = DFSrc[df]()
    colsel(r::NodeRef, idx::Vector{Int}) = Index[idx](r...)
    csvsrc(filen::String, hashdr::Bool) = Alias(dfsrc(CSV.read(filen, DataFrame, header = hashdr)))
    colsplit(ref::NodeRef, idx::Int, yhdr::Set{Symbol}, ex::Set{Symbol}) = ColSplit[idx, yhdr, ex](ref...)
    trainvalsplit(ref::NodeRef, pctval::Float64) = RandRowSplit[pctval](ref...)
    fillmissing(ref::NodeRef, methods::Dict{Symbol,Symbol}) = FillMissing[methods](ref...)
    numerize(ref::NodeRef) = Numerize(ref...)
end
```

We first include the dependent Julia and Julius packages and set up
some basic configurations.

In [ ]:
using GraphEngine: RuleDSL, GraphVM
using Base.CoreLogging
using DataScience, AtomExt, GraphIO
using DataFrames

# turn off informational logging output
disable_logging(CoreLogging.Info)

# extend the number of displayed columns in Jupyter notebooks
ENV["COLUMNS"] = 100

In [ ]:
# the project is used for web UI display
config = RuleDSL.Config(:project => "Titanic");

# start data server for web UI
gss = Dict{String,RuleDSL.AbstractGraphState}()
port = GraphVM.drawdataport()
@async GraphVM.startresponder(gss, port)

The dataset can be loaded by either directly using a url or by providing a CSV source file
via rules defined in the `ds` namespace, which are defined in the `DataScience` package. The
line commented out is a rule to load the same data from a URL.

In [ ]:
rawsrc = RuleDSL.@ref ds.csvsrc("../data/titanic.csv", true; label="raw csv")
# rawsrc = RuleDSL.@ref ds.urlsrc("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv", true; label="raw url")

The very first thing a data scientits may want to do is to get a summary of the dataset.
The follow cell shows how it can be done using the `ds.datasummary` rule in the `DataScience`
package.

In [ ]:
rawsummary = RuleDSL.@ref ds.datasummary(rawsrc; label="data summary")

gs = GraphVM.createlocalgraph(config, RuleDSL.GenericData());
GraphVM.calcfwd!(gs, Set([rawsummary]))

The data summary results can be retrieved using the `GraphVM.getdata` method. The data
cached in individual graph nodes are all vectors, the last argument `1` just refers to the
first and only element of the data summary. Some nodes will have multiple entries in the
data vector.

In [ ]:
RuleDSL.getdata(gs, rawsummary, 1)

We observe that some columns in the raw data set have `missing` values. Data imputation
and cleansing is a common task for building ML models. Julius' `DataScience`
library has provided data imputation methods, which can be easily used by calling the
`fillmissing` rule with the desired imputation method for a each field, i.e., we use
median value of Age of all passenger for any missing Age, and use the mode value
(which is true) for any missing Embarked.

In [ ]:
cleansrc = RuleDSL.@ref ds.fillmissing(
    rawsrc, Dict(:Age => :median, :Embarked => :mode); label="impute missing"
)

After data imputation, we recompute the data summary, showing all the
the `missing` values for both `Age` and `Embarked` features have been populated:

In [ ]:
cleansummary = RuleDSL.@ref ds.datasummary(cleansrc; label="clean summary")
GraphVM.calcfwd!(gs, Set([cleansummary]))
RuleDSL.getdata(gs, cleansummary, 1)

The entire data sourcing and cleansing can be visualized interactively in Julius
convenient web UI by clicking the link below.

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs, port; key="data");
display("image/svg+xml", svg)

### 2. Experiment with multiple Machine Learning models

Julius Graph Engine can easily interop with existing Python, Java, C++ and R libraries via
the generic `Atom` interface

For example, the following rules show how easy it is to access Python ML libraries, such as
`sklearn`, by using the `PyTrain` atom provided in the `DataScience` package. The first
parameter of the `PyTrain` atom is the full name of the Python ML class to use. The second
parameter is a Dictionary with the corresponding parameters/options/arguments of that ML
class.

```julia
@addrules ds begin
    classifiertrain(model::Val{:SVC}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.svm.SVC", options](traindat...)
    classifiertrain(model::Val{:DecisionTree}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.tree.DecisionTreeClassifier", options](traindat...)
    classifiertrain(model::Val{:RandomForest}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.ensemble.RandomForestClassifier", options](traindat...)
    classifiertrain(model::Val{:AdaBoost}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.ensemble.AdaBoostClassifier", options](traindat...)
    classifiertrain(model::Val{:MLPC}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.neural_network.MLPClassifier", options](traindat...)
    classifiertrain(model::Val{:GaussianNB}, options::Dict, traindat::NodeRef) = PyTrain["sklearn.naive_bayes.GaussianNB", options](traindat...)
    classifiertrain(model::Val{:XGBoost}, options::Dict, traindat::NodeRef) = PyTrain["xgboost.XGBClassifier", options](traindat...)
    classifiertrain(model::Symbol, options::Dict, traindat::NodeRef; label = "$model-train") = Alias(classifiertrain(val(model), options, traindat))
end
```

We now proceed to train multiple ML models and compare their in-sample and out-sample
performance using metrics, such as Gini. The ML models are trained to predict the survival
probability of Titanic passengers. We first define the list of models we want to compare and
their hyperparameters.

In [ ]:
models = [
    :DecisionTree => Dict(:min_samples_leaf => 0.1),
    :LogisticRegression => Dict(:solver => "saga", :max_iter => 200),
    :AdaBoost => Dict(),
    :XGBoost => Dict(),
    :GradientBoost => Dict(:min_samples_leaf => 0.1),
    :RandomForest => Dict(:min_samples_leaf => 0.1),
    :GaussianNB => Dict(),
];

The target variable name for ML prediction is given below.

In [ ]:
yname = :Survived;

To divide the input dataset for training and validation, we use the `randrowsel` rule
from the `DataScience` package. The resulting node `valind` contains a `DataFrame` with a
validation index column of `Vector{Bool}` type that randomly indicates wether a row is used
for training or validation. The `1/3` is the percentage of rows that will
be reserved for validation.

In [ ]:
valind = RuleDSL.@ref ds.randrowsel(cleansrc, 1 / 3);

Feature engineering is also supported generically by a rule in `DataScience` package. A feature
set is defined by a 3-element `Tuple`, including the name of the feature set, a Julia `Expr`
providing the transformations for additional features and lastly, the columns to be dropped
 from the feature set. In this particular case, we only drop some columns that should have
 no association with a passenger's suvival probability such as Ticket Id and passenger' name
 and Ids. The Cabin is dropped from the feature set because it has too many missing values.

In [ ]:
# definition of feature engineering: (name, transformations, drop columns)
features = (
    :Original,
    quote end,
    [:Cabin, :Ticket, :PassengerId, :Name],
);

Note that `quote end` is just an empty `Expr` that does not create additional
features from the original data set.

`DataScience.ClassifierSpec` is a `struct` that contains all the information required for
training and validating multiple binary classifiers, it includes the key configurations we
have defined so far. It is more convenient and readable to pass a single
`DataScience.ClassifierSpec` object than five separate parameters. The
`DataScience.ClassifierSpec` is generic, it can be used for any binary classifier
problems and data sets.

In [ ]:
cspec = DataScience.ClassifierSpec(models, cleansrc, yname, valind, features);

Now we can proceed and use the `classifiermetrics` rule to compute in-sample and
out-of-sample metrics for each model. Internally, this rule calls many other rules for each
model and metric, eventually invoking the `classifiertrain` rules mentioned at the beginning
of the section:

In [ ]:
metrics = [:gini, :roc, :accuracyrate, :accuracygraph]
basem = RuleDSL.@ref ds.classifiermetrics(cspec, metrics)
gs2 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
@time GraphVM.calcfwd!(gs2, Set([basem]));

We can retrieve in-sample and out-sample performance metrics, for example, the GINIs:

In [ ]:
giniref = RuleDSL.@ref ds.classifiermetric(cspec, :gini)
gini = GraphVM.getdata(gs2, hash(giniref), 1)
ginidf = DataFrame(model=gini[:InSample][!, :Model], InSample_GINI=gini[:InSample][!, 2], OutSample_GINI=gini[:OutSample][!, 2])

The entire data and logic from can be visualized by clicking on the URL below.

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs2, port; key="ml");
display("image/svg+xml", svg)

## Hyperparameter Tuning

Our Julius Graph Engine also provides a generic rule `hypertune` for hyperparameter tuning
of any ML model. This illustrates the power of rule composition, where a single hypertune
rule can leverage existing rules to search for hyperparmeters of any ML model.

For example, for a given machine learning model, we can select a range for a set of
hyperparameters and easily perform a grid search with respect to a metric:

In [ ]:
ht_1 = RuleDSL.@ref ds.hypertune(cspec, :XGBoost,       Dict(), :gini, :n_estimators => 50:50:200, :learning_rate    => .05:.05:.2);
ht_2 = RuleDSL.@ref ds.hypertune(cspec, :AdaBoost,      Dict(), :gini, :n_estimators => 50:50:200, :learning_rate    => .05:.05:.2);
ht_3 = RuleDSL.@ref ds.hypertune(cspec, :GradientBoost, Dict(), :gini, :n_estimators => 50:50:200, :min_samples_leaf => .05:.05:.2);
ht_4 = RuleDSL.@ref ds.hypertune(cspec, :RandomForest,  Dict(), :gini, :n_estimators => 50:50:200, :min_samples_leaf => .05:.05:.2);

Additional search dimensions can be added to the `ds.hypertune` rule by appending additioanl
pairs of hyperparameter => searchgrid to the end of rule parameter. We can then wrap the
previous nodes in a single node for convenience by means of the `alias` rule wich uses
the `Alias` atom:

In [ ]:
tunings = RuleDSL.@ref ds.alias([ht_1, ht_2, ht_3, ht_4]; label="Hyperparameter Tuning")

Now proceed with the computation of all the defined hyperparameter tunings:

In [ ]:
gs3 = GraphVM.createlocalgraph(config, RuleDSL.GenericData());
@time GraphVM.calcfwd!(gs3, Set([tunings]));

The following cell shows the resulting insample and outsample GINI from the different
hyperparametrs for GradientBoost:

In [ ]:
dat = GraphVM.getdata(gs3, hash(ht_3))
df = deepcopy(dat[1][:, 1:2])
df[!, :InSampleGINI] = dat[1][!, 3]
df[!, :OutSampleGINI] = dat[2][!, 3]
df

Please note that the developer is in charge of selecting the optimal hyperparameter set,
as this may involve multiple objectives, the parameter set producing the maximum out-sample
gini may not be the best choice. Often, it is better to choose the parameter set with similar
in-sample and out-of-sample gini to minimize the chance of overfitting.

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs3, port; key="hyper");
display("image/svg+xml", svg)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*